In [1]:
import mujoco, numpy as np, time
import mujoco.viewer

m = mujoco.MjModel.from_xml_path("resizable_box.xml")
d = mujoco.MjData(m)

bid  = mujoco.mj_name2id(m, mujoco.mjtObj.mjOBJ_BODY,  "box")
gid  = mujoco.mj_name2id(m, mujoco.mjtObj.mjOBJ_GEOM,  "gbox")
jxid = mujoco.mj_name2id(m, mujoco.mjtObj.mjOBJ_JOINT, "ux_joint")
jyid = mujoco.mj_name2id(m, mujoco.mjtObj.mjOBJ_JOINT, "uy_joint")
jzid = mujoco.mj_name2id(m, mujoco.mjtObj.mjOBJ_JOINT, "uz_joint")
qx, qy, qz = m.jnt_qposadr[jxid], m.jnt_qposadr[jyid], m.jnt_qposadr[jzid]

free = next(i for i in range(m.njnt) if m.jnt_type[i]==mujoco.mjtJoint.mjJNT_FREE)
free_q = m.jnt_qposadr[free]

M    = 1.0
EPS  = 1e-3   # min half-extent

def apply_from_joints():
    ux = max(float(d.qpos[qx]), EPS)
    uy = max(float(d.qpos[qy]), EPS)
    uz = max(float(d.qpos[qz]), EPS)

    # resize COLLISION + visual geom
    m.geom_size[gid, :] = [ux, uy, uz]

    # solid box inertia with full lengths (2ux,2uy,2uz)
    Ixx = (M/3.0)*(uy*uy + uz*uz)
    Iyy = (M/3.0)*(ux*ux + uz*uz)
    Izz = (M/3.0)*(ux*ux + uy*uy)
    m.body_mass[bid]       = M
    m.body_inertia[bid, :] = [Ixx, Iyy, Izz]

    # keep it above the plane so it never starts intersecting
    d.qpos[free_q + 2] = max(d.qpos[free_q + 2], uz + 1e-4)
    mujoco.mj_forward(m, d)

# pick initial “default” extents via controls (or set qpos directly)
d.ctrl[:] = [0.30, 0.25, 0.40]
d.qpos[2] = 2
apply_from_joints()

with mujoco.viewer.launch_passive(m, d) as v:
    while v.is_running():
        apply_from_joints()
        mujoco.mj_step(m, d)
        v.sync()
        time.sleep(m.opt.timestep)


/home/yugiero/tma_env/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65548) b'Wayland: The platform does not provide the window position'
  warnings.warn(message, GLFWError)
